In [2]:
!pip install git+https://github.com/openai/swarm.git --quiet
!pip install ollama openai --quiet
# !pip install -r requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip


In [3]:

# !pip install tiktoken==0.8.0 openai==1.59.5 chromadb==0.6.2 FlagEmbedding==1.3.3 pymupdf4llm==0.0.17 tenacity==9.0.0 ollama


In [1]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [1]:
from chromadb.utils import embedding_functions
from FlagEmbedding import FlagReranker
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from FlagEmbedding import BGEM3FlagModel
import pymupdf4llm
import logging
from tqdm import tqdm
import numpy as np

logger = logging.getLogger(__name__)


/workspace/ollama-pydantic/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:

def embedding_function_bge(text_list):
    return emb_model.encode(text_list, return_dense=True)['dense_vecs']



class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        embeddings = embedding_function_bge(input)
        return embeddings

emb_model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)
default_ef = MyEmbeddingFunction()
client = chromadb.PersistentClient(path="chromadb_folder")

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 165347.07it/s]


In [3]:
embedding_function_bge(['hi there'])

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


array([[-0.04208 , -0.001314, -0.05435 , ...,  0.007812, -0.02707 ,
         0.0521  ]], shape=(1, 1024), dtype=float16)

In [4]:
# from ollama import AsyncClient
# ollama_client = AsyncClient()



In [4]:
import os
os.environ['OPENAI_BASE_URL']='http://localhost:11434/v1' 
# os.environ['SWARM_DEFAULT_MODEL']='llama3-groq-tool-use'
os.environ['OPENAI_API_KEY']='ollama'


# model = 'llama3-groq-tool-use'
# model = 'llama3.2:3b-instruct-fp16'
# model = 'llama3.1:8b'
# model = 'qwen2.5:14b'
model = 'qwen2.5:14b'

In [5]:

def process_texts(texts, chunk_size=100, overlap=30):
    """Process a list of texts, splitting them into chunks of specified size with overlap,
    and accumulating shorter texts."""
    accumulated_words = []  # Accumulate words from texts shorter than chunk_size
    final_chunks = []  # Store the final chunks of text

    for text in texts.split():
        accumulated_words.append(text)

        while len(accumulated_words) >= chunk_size:
            # Take the first chunk_size words for the current chunk
            chunk = " ".join(accumulated_words[:chunk_size])
            final_chunks.append(chunk)
            # Remove words from the start of the accumulated_words, considering overlap
            accumulated_words = accumulated_words[chunk_size - overlap:]

    # If there are any remaining words, form the last chunk
    if accumulated_words:
        final_chunks.append(" ".join(accumulated_words))

    return final_chunks

def get_unique_text_indices(text_list):
    unique_texts = {}
    unique_indices = []

    for i, text in enumerate(text_list):
        if text not in unique_texts:
            unique_texts[text] = i
            unique_indices.append(i)

    return unique_indices

def create_pdf_collection(pdf_path):
    """
    Process a PDF file and add its chunks to a collection.

    Args:
        pdf_path: Path to the PDF file
        collection: The collection object to add documents to
    """
    try:
        print('Create collection Running....')
        md_text = pymupdf4llm.to_markdown(pdf_path,show_progress=False)
        all_chunks = process_texts(md_text, chunk_size=500, overlap=50)
        collection = client.get_or_create_collection(name='database',embedding_function=default_ef)
        logger.info(collection)

        for idx, chunk in tqdm(enumerate(all_chunks)):
            id_ = str(idx)
            collection.add(
                documents=[chunk],
                ids=[id_]
            )
        status = 'success'
        print('Done.')
        return status

    except Exception as e:
        logger.error(f"Error creating pdf collection: {str(e)}", exc_info=True)
        return f'Sorry for inconvenience. Error creating pdf collection. Please contact support.'


def get_full_context(query, collection, n_results=5, top=2):

    ''''
    Get the context from database for given query.

    Args:
        query: Query string
        collection_name: Database collection object
        n_results: Number of results to retrieve
        top: Number of top results to return
    Returns:
          str: context based on query.
    '''

    logger.info(f'quering collection---> {collection}')

    result = collection.query(query_texts = query,n_results=n_results)
    texts = result['documents'][0]
    ids = result['ids'][0]
    unique_indices = get_unique_text_indices(texts)
    unique_docs = [texts[x] for x in unique_indices]
    unique_ids = [ids[x] for x in unique_indices]
    ## colbert
    query_col = emb_model.encode([query],return_colbert_vecs=True)
    docs_col = emb_model.encode(unique_docs,return_colbert_vecs=True)
    colber_scores = []
    for vectors in docs_col['colbert_vecs']:
        colber_scores.append(emb_model.colbert_score(query_col['colbert_vecs'][0],vectors).numpy())

    ## full_context_colbert
    full_context_scores = []
    full_context_ids = []
    for id in unique_ids:
        pre_id,post_id = str(int(id)-1), str(int(id)+1)
        # print(pre_id,id,post_id)
        full_context_ids.append([pre_id,id,post_id])
        full_context=collection.get(ids=[f'{pre_id}',f'{id}',f'{post_id}'])['documents']
        full_context = ''.join(full_context)
        full_context_colber_vec = emb_model.encode([full_context],return_colbert_vecs=True)
        full_context_colber_score = emb_model.colbert_score(query_col['colbert_vecs'][0],full_context_colber_vec['colbert_vecs'][0]).numpy()

        full_context_scores.append(full_context_colber_score)

    all_scores = [2*full_context_scores[i]+0.9*colber_scores[i] for i in range(len(colber_scores))]
    sorted_indices = [index for index, _ in sorted(enumerate(all_scores), key=lambda x: x[1], reverse=True)]
    top_context_ids_list = [full_context_ids[index] for index in sorted_indices][:top]
    flattened_list = np.array(top_context_ids_list).flatten().tolist()
    top_ids = list(set(flattened_list))
    top_ids.sort()
    top_context = collection.get(ids=top_ids)['documents']
    # top_context = ''.join(top_context)

    logger.info(f'context retrieved from collection---> {collection}')
    return top_context,top_ids


In [6]:
# create_pdf_collection('/workspace/ollama-pydantic/llama3-report.pdf')

In [10]:


# client = Client()
# response = client.create(
#   model='my-assistant',
#   from_='llama3.2',
#   system='You are mario from Super Mario Bros.',
#   stream=False,
# )
# print(response.status)

success


In [18]:
def get_context_based_on_collection(query):
  '''
  Input: user query
  Return the query answer based on context available in database.
  '''
  collection = client.get_or_create_collection(name='database',embedding_function=default_ef)

  context, context_idx = get_full_context(query, collection, n_results=5, top=2)

  if context:
    print('context found')

    return ' '.join(context)
  else:
    return f'No context found for given {query} through collection'


context found


In [17]:
# context

query = 'trainign methods involved'
context = get_context_based_on_collection(query)
def get_answer(context,query):
    # from ollama import AsyncClient
    from ollama import Client
    messages = [
        {
          'content': f'Based on given context: {context}, answer this query: {query} in full detail. always answer in full detailed and pointers. if context is irrelevant, say no relevant context found to answer query.',
          'role': 'user',
        },
      ]
    
    ollama_client = Client()
    response = ollama_client.chat(model, messages=messages)
    return response['message']['content']

In [19]:
get_answer(context,query)

"The training process for developing Llama 3 involves several comprehensive stages designed to enhance the model's performance, alignment with human preferences, and robustness across various tasks. Below is a detailed breakdown of each stage:\n\n### 1. **Reward Model Training**\n   - **Objective:** To create a reward model that can evaluate generated responses based on human preferences.\n   - **Data Preparation:**\n     - Preference data consists of triplets containing edited response, chosen response, and rejected response in clear ranking order (edited > chosen > rejected).\n     - Triplets are constructed to ensure the reward model learns from human judgments effectively.\n   - **Training Process:**\n     - The training involves concatenating prompts with randomly shuffled responses into single rows for a more comprehensive understanding of the context.\n     - This step trains a neural network that can predict which response is preferred based on its alignment with human preferen

In [93]:
# from swarm import Swarm, Agent


# Swarm_client = Swarm()

# english_agent = Agent(
#     name="English Agent",
#     instructions="You only speak English.",
#     model = model
# )

# spanish_agent = Agent(
#     name="Spanish Agent",
#     instructions="You only speak Spanish.",
#     model = model
# )


# def transfer_to_spanish_agent():
#     """Transfer spanish speaking users immediately."""
#     return spanish_agent


# english_agent.functions.append(transfer_to_spanish_agent)

# messages = [{"role": "user", "content": "Hola. ¿Como estás?"}]
# response = client.run(agent=english_agent, messages=messages)

# print(response.messages[-1]["content"])


In [20]:
from pydantic import BaseModel
from typing import Optional
class Agent(BaseModel):
    name: str = "Agent"
    model: str = "gpt-4o-mini"
    instructions: str = "You are a helpful Agent"
    tools: list = []

class Response(BaseModel):
    agent: Optional[Agent]
    messages: list


def execute_tool_call(tool_call, tools, agent_name):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"{agent_name}:", f"{name}({args})")

    return tools[name](**args)


from ollama import AsyncClient

ollama_client = AsyncClient()

# messages = [
#     {
#       'content': f'Based on given context: {context}, answer this query: {query} in full detail. always answer in full detailed and pointers. if context is irrelevant, say no relevant context found to answer query.',
#       'role': 'user',
#     },
#   ]


# response = await ollama_client.chat(model, messages=messages)
# print(response['message']['content'])

In [140]:
def transfer_to_answer_agent(**args):
  '''
  Transfer session to answering agent.

  '''
  return answer_agent


def transfer_to_main_agent(**args):
  '''
  Transfer session to main agent.

  '''
  return main_agent

main_agent = Agent(
    model=model,
    name="Main Agent",
    instructions='''
You are the Main Agent.

### DUTIES:
1. **User Interaction & Greeting**:
   - Always greet the user during the first interaction.
   - Clearly state your purpose as the Main Agent.
  
2. **Providing Responses**:
   - Ask user for query regarding internal database only.
   - Transfer session to answering agent 'transfer_to_answer_agent'
   - Summarize the findings of the Answering Agent concisely and accurately.
   - Do not provide additional interpretations or assumptions beyond what the Answering Agent provides.
   - 

3. **Role Adherence**:
   - Do not assume any role other than that of a helpful assistant.
   - Do not generate responses independently; rely solely on the Answering Agent’s findings.

''',
    tools=[transfer_to_answer_agent],

)

answer_agent = Agent(
    model=model,
    name="Answering Agent",
    instructions='''
You are the Answering Agent, responsible for retrieving information from the internal database and responding to user queries.

### DUTIES:
1. **Query Handling**:
   - Answer user queries using the available database.
   - Retrieve relevant context using the provided function `get_context_based_on_collection`.

2. **Function Usage**:
   - Use `get_context_based_on_collection` to gather context around the user query before responding.

3. **Role Adherence**:
   - Ensure all responses are based solely on the available database—do not generate assumptions or external knowledge.
   - if user end with ok, thanks , and other experession to conclude, then conclude the session


''',
    tools=[get_context_based_on_collection,transfer_to_main_agent],
    # tool_choice='auto'
)


In [132]:
import inspect
import json
def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

def sample_function(param_1, param_2, the_third_one: int, some_optional="John Doe"):
    """
    This is my docstring. Call this function when you want.
    """
    print("Hello, world")

schema =  function_to_schema(sample_function)
print(json.dumps(schema, indent=2))

{
  "type": "function",
  "function": {
    "name": "sample_function",
    "description": "This is my docstring. Call this function when you want.",
    "parameters": {
      "type": "object",
      "properties": {
        "param_1": {
          "type": "string"
        },
        "param_2": {
          "type": "string"
        },
        "the_third_one": {
          "type": "integer"
        },
        "some_optional": {
          "type": "string"
        }
      },
      "required": [
        "param_1",
        "param_2",
        "the_third_one"
      ]
    }
  }
}


In [133]:
# ollama_client = AsyncClient()

# messages = [
#     {
#       'content': f'transfer to main agent',
#       'role': 'user',
#     },
#   ]


# response = await ollama_client.chat(model, messages=messages,tools = [transfer_to_main_agent])
# print(response['message']['content'])

In [134]:
import uuid
uuid.uuid4()

UUID('52ab408c-ea9a-4783-9960-3c0a5f080560')

In [135]:
async def run_full_turn(agent, messages):

    current_agent = agent
    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in current_agent.tools]
        tools = {tool.__name__: tool for tool in current_agent.tools}

        # === 1. get openai completion ===
        response = await ollama_client.chat(agent.model,  messages=[{"role": "system", "content": current_agent.instructions}]
            + messages,tools=tool_schemas)
        message = response['message']
        messages.append(response.message)

        if message.content:  # print agent response
            print(f"{current_agent.name}:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools, current_agent.name)

            if type(result) is Agent:  # if agent transfer, update current agent
                current_agent = result
                result = (
                    f"Transfered to {current_agent.name}. Adopt persona immediately."
                )

            result_message = {
                "role": "tool",
                "tool_call_id": uuid.uuid4(),
                "content": result,
                'name': tool_call.function.name
            }
            messages.append(result_message)

    # ==== 3. return last agent used and new messages =====
    return Response(agent=current_agent, messages=messages[num_init_messages:])


def execute_tool_call(tool_call, tools, agent_name):
    name = tool_call.function.name
    args = tool_call.function.arguments

    print(f"{agent_name}:", f"{name}({args})")

    return tools[name](**args)  # call corresponding function with provided arguments

In [141]:
agent = main_agent
messages = []

while True:
    user = input("User: ").strip()
    if user.lower()=='q':
      break
    messages.append({"role": "user", "content": user})

    response = await run_full_turn(agent, messages)
    agent = response.agent
    messages.extend(response.messages)

User:  hi


Main Agent: transfer_to_answer_agent({'args': ''})
Answering Agent: Hello! How can I assist you today?


User:  q


In [99]:
# agent = main_agent
# messages = []

# while True:
#     user_msg = input("User: ").strip()
#     if user_msg.lower()=='q':
#       break
#     user_template = [{"role": "user", "content": f"{user_msg}"}]
#     messages.extend(user_template)
#     response = Swarm_client.run(agent=agent,messages = messages)
#     response_content = response.messages[-1]["content"]
#     print(f'{response.agent.name}: {response_content}')
#     agent = response.agent
#     messages.extend(response.messages)


In [142]:
# messages


In [ ]:
from transformers import AutoModel

model_name = "BAAI/bge-base-en-v1.5"  # Replace with the correct model ID
save_path = "./models/bge-m3"

# Download and save the model locally
model = AutoModel.from_pretrained(model_name)
model.save_pretrained(save_path)

print("Model downloaded and saved locally at:", save_path)
